## 作用簡介
這個 notebook (Loading_All_Documents.ipynb) 是用來:
- 自動讀取目錄中的文件 (txt)
- 分段處理 大文本，分成小 chunk
- 用 OpenAI 生成向量形式 (embedding)
- 將向量儲存到 ChromaDB (可持久化)

用白話說，就是：

- 讀取一堆 .txt 文件資料 ➔ 切成小段（chunk） ➔ 把小段轉成向量（embedding） ➔ 存到 ChromaDB
- 當有提問時（query），從 ChromaDB 撈最相關的段落 ➔ 把找到的段落餵給 OpenAI 的 GPT 模型，生成回答

##  安裝清單
```python
pip install chromadb
pip install python-dotenv
pip install openai
```


## 前置作業

In [2]:
import os
from dotenv import load_dotenv
import chromadb
from openai import OpenAI
from chromadb.utils import embedding_functions

# 載入 .env 檔案中的環境變數
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_key)

# 設定 OpenAI 的 Embedding Function，使用 text-embedding-3-small 模型
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key, model_name="text-embedding-3-small"
)

# 初始化 ChromaDB，並設定持久化儲存路徑
chroma_client = chromadb.PersistentClient(path="./db/chroma_persistent_storage")
collection_name = "document_qa_collection"

# 建立或取得一個 collection，並指定使用的 embedding function
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)

# 初始化 OpenAI 的客戶端
client = OpenAI(api_key=openai_key)



## 初始設定區（第一次執行才要打開，建完資料庫後可以註解掉）

In [3]:
# 從資料夾中讀取所有 .txt 文件
def load_documents_from_directory(directory_path):
    print("==== 正在從資料夾載入文件 ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents

# 將文字切成小段落（chunk）
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap  # 讓段落之間有重疊，避免斷句影響理解
    return chunks

# 載入資料夾內所有文件
directory_path = "../測試RAG文章/"
documents = load_documents_from_directory(directory_path)

# 將文件內容切成小段
chunked_documents = []
print("==== 正在切割文件為小段 ====")
for doc in documents:
    chunks = split_text(doc["text"])
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})


# 透過 OpenAI API 生成文本的向量（Embedding）
def get_openai_embedding(text):
    print("==== 正在生成 Embedding ====")
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    return embedding

# 為每個小段文字生成對應的向量
print("==== 正在產生每個段落的向量 ====")
for doc in chunked_documents:
    doc["embedding"] = get_openai_embedding(doc["text"])

# 將段落和對應向量寫入到 Chroma 資料庫
for doc in chunked_documents:
    print("==== 正在把段落寫入 Chroma 資料庫 ====")
    collection.upsert(
        ids=[doc["id"]], documents=[doc["text"]], embeddings=[doc["embedding"]]
    )

==== 正在從資料夾載入文件 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在切割文件為小段 ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding ====
==== 正在產生每個段落的向量 ====
==== 正在生成 Embedding 

## 每一次做查詢

In [4]:
# 查詢資料庫的函式
def query_documents(question, n_results=2):
    # 用問題文字去資料庫查找最相關的段落
    results = collection.query(query_texts=question, n_results=n_results)

    # 把找到的段落提取出來
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    print("==== 回傳找到的相關段落 ====")
    return relevant_chunks

# 產生回覆的函式，利用找到的段落讓 GPT 回答問題
def generate_response(question, relevant_chunks):
    # 把找到的段落串成一個 context
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "你是一個用來回答問題的小助手。請根據下列提供的內容來回答問題，"
        "如果不知道答案，請直接說不知道。回答請控制在三句話以內，並保持簡潔。"
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    # 呼叫 OpenAI 的聊天 API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )

    answer = response.choices[0].message
    return answer


# 測試：提出一個問題並取得回答
question = "請幫我簡單總結一下這些文章的內容。"
relevant_chunks = query_documents(question)
answer = generate_response(question, relevant_chunks)

print("==== 答案如下 ====")
print(answer.content)


==== 回傳找到的相關段落 ====
==== 答案如下 ====
這篇文章主要介紹了Bing Chat的新功能，包括透明度提升、引用及視覺化數據呈現能力。另外，也提到了谷歌在人工智慧領域的發展，包括深度學習和語言模型。
